# My TCP/IP

In [4]:
import socket

## Raw socket

In order to put data on the wire, we need to open a raw socket.

Opening a raw socket is usually restricted by the operating system for security reasons. If a user can open a raw socket then he can snoop on every packet received on that network interface, including packets destined for other users! So raw sockets are only available to root.

On Linux, permission can be given to individual programs by setting *capabilities*. Since we are using Python, that means giving this Python interpreter the capabilities. I created this virtualenv using the `--copies` option, which makes a copy of the interpreter instead of a symlink. Then:

```sh
sudo setcap cap_net_admin,cap_net_raw=eip /path/to/venv/python
```

Now we can write a function send arbitrary data using a raw socket. `AF_PACKET` means we want to bind to a link-level interface directory. `SOCK_RAW` means we are opening a raw socket.

In [47]:
def send_bytes(interface_name, data):
    s = socket.socket(socket.AF_PACKET, socket.SOCK_RAW)
    s.bind((interface_name, 0))
    s.send(data)
    s.close()

But we can't just send arbitrary data, it must be a valid Ethernet frame:

In [13]:
try:
    send_bytes("lo", b"HELLO")
except OSError:
    print("Invalid Ethernet frame!")

Invalid Ethernet frame!


## OSI model layers

1. Physical (Copper, Fibre, Radio etc.)
2. Link (Ethernet, PPP etc.)
3. Network (IPv4, IPv6)
4. Transport (TCP, UDP)
5. Session
6. Presentation
7. Application (HTTP etc.)

## Ethernet (Layer 2)

Ethernet supports sending frames. It can be assumed that all frames sent will be received by all other interfaces on the network. All interfaces are uniquely identified by a MAC address and each frame will contain a source and destination MAC address. This way, interfaces can decide which frames are intended for them and which are not.

### MAC addresses

MAC addresses are 48 bits (6 bytes) long. They are usually formatted in hexadecimal with bytes separated by colons.

In [72]:
def encode_mac_addr(mac_address):
    return bytes.fromhex(mac_address.replace(":", ""))

In [63]:
encode_mac_addr("01:02:03:04:05:06")

b'\x01\x02\x03\x04\x05\x06'

In [65]:
def format_mac_addr(mac_address):
    return ":".join(f"{byte:02x}" for byte in mac_address)

In [67]:
format_mac_addr(b"\x01\x02\x03\x04\x05\x06")

'01:02:03:04:05:06'

### Ethernet II frame

In addition to the MAC addresses, an Ethernet II frame only contains a 2-byte *ethertype*, the payload, and a checksum. The checksum will be automatically generated by the operating system, so we only need to specify the others:

In [43]:
def ethernet_frame(dst_addr, src_addr, ethertype, payload):
    return dst_addr + src_addr + ethertype + payload

Some common ethertypes are:

In [74]:
ARP  = bytes.fromhex("08 06")
IPV4 = bytes.fromhex("08 00")
IPV6 = bytes.fromhex("86 DD")
CUST = bytes.fromhex("08 01")  # custom, not a real protocol

So we can now send a frame to the loopback interface:

In [68]:
frame = ethernet_frame(
    src_addr=encode_mac_addr("00:00:00:00:00:00"),
    dst_addr=encode_mac_addr("00:00:00:00:00:00"),
    ethertype=CUST,
    payload="Hello, world!".encode(),
)

send_bytes("lo", frame)

## Internet Protocol version 4

Ethernet is suitable for small to medium sized networks but it could never work on the scale of the Internet. The Internet Protocol version 4 (IPv4) is used to route packets across many different layer 1/2 networks such that any two devices in the world can communicate.

To achieve this, each device gets an IP address in addition to its MAC address. Unlike MAC addresses, which are hardwired into network interfaces, IP addresses are dynamic and only make sense in layer 3.

In addition to an IP address, each device is also configured with a subnet mask and a gateway. The subnet mask tells the host which other IP addresses are on its local network. For those on its local network, communication is performed directly over Ethernet. For hosts outside of its local network, packets are instead sent to the router which will forward it to another network which contains that host.

For example, a host can be configured like this:

```
Address:     10.0.0.1 (or 10.0.0.1/24)
Subnet Mask: 255.255.255.0
Gateway:     10.0.0.254
```

The host `10.0.0.2` is on the same network and should be reachable on the layer 1/2 network, but the host `10.0.1.1` is on another network and will only be reachable via the gateway at `10.0.0.254`.

In [57]:
def encode_ip4_addr(ip4_addr):
    return bytes(int(s) for s in ip4_addr.split("."))

In [59]:
encode_ip4_addr("10.0.0.254")

b'\n\x00\x00\xfe'

In [60]:
def decode_ip4_addr(ip4_addr):
    return ".".join(f"{byte:d}" for byte in ip4_addr)

In [61]:
decode_ip4_addr(b'\n\x00\x00\xfe')

'10.0.0.254'

### Address Resolution Protocol (Layer 2)

If we somehow know the IP address of a device on our local network, we need to know its MAC address to be able to communicate. ARP is used to resolve IP addresses into MAC addresses. ARP is not specific to IPv4 or Ethernet and supports different address sizes, but we will keep it specific to IPv4 over Ethernet.

In [69]:
def arp_request(src_mac_addr, src_ip_addr, dst_ip_addr):
    hardware_type = bytes.fromhex("00 01")  # Ethernet
    protocol_type = bytes.fromhex("08 00")  # IPv4
    hardware_len  = bytes.fromhex("06")     # MAC address length
    protocol_len  = bytes.fromhex("04")     # IPv4 address length
    operation     = bytes.fromhex("00 01")  # request
    dst_mac_addr  = encode_mac_addr("00:00:00:00:00:00")  # ignored for a request
    
    return (
        hardware_type
        + protocol_type
        + hardware_len
        + protocol_len
        + operation
        + src_mac_addr
        + src_ip_addr
        + dst_mac_addr
        + dst_ip_addr
    )

If we want to receive a reply to an ARP request, we must put in our real MAC address (this is technically only necessary if the network uses switches instead of hubs). We will send a MAC request to the router on this network.

In [73]:
MY_MAC_ADDR = encode_mac_addr("1c:87:2c:46:e0:47")
MY_IP_ADDR = encode_ip4_addr("192.168.1.250")
ROUTER_IP_ADDR = encode_ip4_addr("192.168.1.1")

arp_packet = arp_request(
    src_mac_addr=MY_MAC_ADDR,
    src_ip_addr=MY_IP_ADDR,
    dst_ip_addr=ROUTER_IP_ADDR,
)

When we send this packet, we want to broadcast it to the network (since we don't know the MAC address of the router yet. To do that, a special MAC address `ff:ff:ff:ff:ff:ff` is used.

In [79]:
BROADCAST_MAC_ADDR = encode_mac_addr("ff:ff:ff:ff:ff:ff")
INTERFACE_NAME = "eno1"

send_bytes(
    INTERFACE_NAME,
    ethernet_frame(
        src_addr=MY_MAC_ADDR,
        dst_addr=BROADCAST_MAC_ADDR,
        ethertype=ARP,
        payload=arp_packet,
    ),
)

After sending this packet to the network we would expect to receive a reply from the router telling us its MAC address. This can be stored in a table for later use.

In [80]:
ARP_TABLE = {
    "192.168.1.250": "00:1f:16:f8:37:86",
}

# IP (Layer 3)

Now that we can resolve IP addresses to MAC addresses we can build IP packets to send across networks.

In [108]:
def calculate_checksum(header):
    words = [int.from_bytes(header[i:i+2], "big") for i in range(0, len(header), 2)]
    full_sum = sum(words)
    overflow = full_sum >> 16
    full_sum = (full_sum&0xFFFF) + overflow
    overflow = full_sum >> 15
    full_sum = (full_sum&0xFFFF) + overflow
    return (~full_sum&0xFFFF)

In [83]:
def ip4_packet(src_ip_addr, dst_ip_addr, protocol, payload):
    version = 4
    ihl = 5  # header length in 32 bits words
    dscp = 0
    ecn = 0
    total_length = ihl*4 + len(payload)
    identification = 0  # only important for fragmentation
    flags = 0b010  # don't fragment
    fragment_offset = 0
    ttl = 64
    
    header = bytearray(ihl*4)
    header[0] = (version<<4) + ihl
    header[1] = (dscp<<2) + ecn
    header[2:4] = total_length.to_bytes(2, "big")
    header[4:6] = identification.to_bytes(2, "big")
    header[6:8] = ((flags<<13) + fragment_offset).to_bytes(2, "big")
    header[8] = ttl
    header[9] = protocol
    header[12:16] = src_ip_addr
    header[16:20] = dst_ip_addr
    
    checksum = calculate_checksum(header)
    header[10:12] = checksum.to_bytes(2, "big")
    
    return header + payload

In [112]:
send_bytes(
    "lo",
    ethernet_frame(
        src_addr=MY_MAC_ADDR,
        dst_addr=BROADCAST_MAC_ADDR,
        ethertype=IPV4,
        payload=ip4_packet(
            src_ip_addr=encode_ip4_addr("192.168.1.250"),
            dst_ip_addr=encode_ip4_addr("192.168.1.1"),
            protocol=0xfd,
            payload=b"Hello, world!",
        ),
    ),
)